In [1]:
!pip install accelerate
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken

In [2]:
!pip install docx2txt

In [3]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders.word_document import Docx2txtLoader
import re

In [4]:
def embed_model():
  model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
  device = "cuda" if torch.cuda.is_available() else "cpu"
  return HuggingFaceEmbeddings(model_name = model, model_kwargs = {"device": device})

In [5]:
def llm_model(load_in_8bit):
  model = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"
  tokenizer = AutoTokenizer.from_pretrained(model)
  return pipeline(
      task = "text2text-generation",
      model = model,
      tokenizer = tokenizer,
      max_new_tokens = 100,
      device_map = "auto",
      model_kwargs = {
          "load_in_8bit": load_in_8bit
      }
  )

In [6]:
embedding = embed_model()
llm = llm_model(load_in_8bit = False)

/tmp/ipython-input-4-2050498128.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name = model, model_kwargs = {"device": device})


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['PeftModelForSeq2SeqLM', 'BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'GraniteSpeechForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'T

In [23]:
docx_path = "/content/itapersian.docx"
loader = Docx2txtLoader(docx_path)
documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size = 1000, chunk_overlap = 200)
texts = text_splitter.split_documents(documents)
persist_directory = "./"
vectordb = Chroma.from_documents(documents = texts, embedding = embedding, persist_directory = persist_directory)

In [8]:
_question = "تلاش ایران برای حضور در جام جهانی فوتبال از چه سالی آغاز شد؟"
prompt = "به پرسش زیر پاسخ بده. \n" + _question
result = llm(prompt)
print(result)

[{'generated_text': 'به پرسش زیر پاسخ بده. \nتلاش ایران برای حضور در جام جهانی فوتبال از چه سالی آغاز شد؟\n\nپاسخ: تلاش ایران برای حضور در جام جهانی فوتبال از سال 1950 آغاز شد. ایران در سال 1948 به فدراسیون بین\u200cالمللی فوتبال (فیفا) پیوست و در سال 1950 برای اولین بار به جام جهانی فوتبال راه یافت. آنها در جام جهانی 1950 در برزیل شرکت کردند و در گروه خود با برزیل، سوئد و پاراگوئه قرار گرفتند. آنها در سه بازی خود با بر'}]


In [9]:
_question = "کدام مربی نتوانست ایران را به جام جهانی 2002 برساند؟"
prompt = "به پرسش زیر پاسخ بده. \n" + _question
result = llm(prompt)
print(result)

[{'generated_text': 'به پرسش زیر پاسخ بده. \nکدام مربی نتوانست ایران را به جام جهانی 2002 برساند؟ \n1. برانکو ایوانکوویچ \n2. علی دایی \n3. جوزپه سکی \n4. کارلوس Alberto Parreira \n\nپاسخ: 1. برانکو ایوانکوویچ \n\nبرانکو ایوانکوویچ مربی تیم ملی فوتبال ایران بود که در جام جهانی 2002 شرکت کرد، اما نتوانست ایران را به جام جهانی 2002 برساند. علی دایی، جوزپ'}]


In [10]:
_question = "ایران در بازی نخست مقابل مراکش چند امتیاز کسب کرد؟"
prompt = "به پرسش زیر پاسخ بده. \n" + _question
result = llm(prompt)
res = result[0]['generated_text']
res = re.sub("\u200c", " ", res)
res = re.sub("\n", " ", res)
print(res)

به پرسش زیر پاسخ بده.  ایران در بازی نخست مقابل مراکش چند امتیاز کسب کرد؟  ایران در بازی نخست مقابل مراکش 1 امتیاز کسب کرد.


In [24]:
hf_llm = HuggingFacePipeline(pipeline = llm)
retriever = vectordb.as_retriever(search_kwargs = {"k": 1})
qa = RetrievalQA.from_chain_type(llm = hf_llm, chain_type = "stuff", retriever = retriever)
question_template = """
با توجه به متن زیر به پرسش پاسخ بده:
متن: {context}
پرسش: {question}
پاسخ را به زبان فارسی روان بده:
"""
QUESTION_PROMPT = PromptTemplate(
    template = question_template, input_variables = ["context", "question"]
)
qa.combine_documents_chain.llm_chain.prompt = QUESTION_PROMPT

In [25]:
len(texts)

8

In [26]:
for t in texts[:2]:
  print(t)
  print("#" * 80)

page_content='پیام رسان ایتا به عنوان یکی از اپلیکیشن‌های ارتباطی ایرانی در سال ۱۳۹۷ راه اندازی شد. هدف اصلی این نرم افزار، پاسخگویی به نیازهای کاربران ایرانی در زمینه ارتباطات سریع و امن بوده است. ایتا به کاربران این امکان را میدهد که به صورت فردی یا گروهی با یکدیگر ارتباط برقرار کنند و محتوای مختلفی از جمله متن، تصویر و ویدیو را به اشتراک بگذارند. با توجه به محدودیت‌های ناشی از تحریم ها و نیاز به نرم افزارهای بومی، ایتا به عنوان یک جایگزین معتبر برای پیام رسان‌های خارجی به شمار می آید.

این نرم افزار در راستای ارتقاء ارتباطات داخلی و کاهش وابستگی به پیام رسان‌های خارجی طراحی شده است. با توجه به نیاز روزافزون جامعه به پلتفرم‌های ارتباطی امن و قابل اعتماد، ایتا به سرعت در میان کاربران ایرانی محبوبیت پیدا کرد. به علاوه، سیاست های حریم خصوصی ایتا به وضوح اعلام شده و این موضوع به ایجاد اعتماد در میان کاربران کمک کرده است.

یکی از بزرگترین مزایای ایتا، توجه به حریم �' metadata={'source': '/content/itapersian.docx'}
###########################################################################

In [31]:
_question = "ایتا به صورت کامل به چه زبانی طراحی شده است؟"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
res2 = qa({"query": _question,})['result']
res2 = re.sub("\u200c", " ", res2)
res2 = re.sub("\n", " ", res2)
res2 = re.sub("\xa0", " ", res2)
print(res2)



> Entering new StuffDocumentsChain chain...

> Finished chain.
 با توجه به متن زیر به پرسش پاسخ بده: متن: ف دسترسی پیدا کنند.  ایتا امکاناتی نظیر ایجاد گروه، کانال، ارسال استیکر و فایل های چندرسانه ای را ارائه میدهد. همچنین، کاربران میتوانند از قابلیت های صوتی و تصویری نیز بهره مند شوند. این ویژگی ها باعث میشود که ایتا یک پلتفرم جامع برای ارتباطات روزمره باشد.  ایتا به صورت کامل به زبان فارسی طراحی شده است، که این امر به کاربران ایرانی کمک میکند تا به راحتی از امکانات آن استفاده کنند و نیاز به یادگیری زبان های دیگر نداشته باشند. این موضوع به ویژه برای کاربران غیرحرفه ای و سالمندان که ممکن است با زبان های خارجی آشنایی نداشته باشند، مزیت بزرگی به شمار می آید.  ایتا به امنیت اطلاعات کاربران توجه ویژه ای دارد. با استفاده از رمزنگاری پیشرفته، اطلاعات منتقل شده بین کاربران به خوبی محافظت می شود. این ویژگی امنیتی برای کاربران مهمی است که به دنبال یک پلتفرم امن برای ارتباطات خود هستند.  ایتا به عنوان یک پیام رسان بومی، قابلیت توسعه و به روز رسانی های سریعتر نسبت به پرسش: ایتا به صورت کامل به

In [28]:
_question = "یکی از یزرگ ترین مزایای ایتا چیست؟"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
res2 = qa({"query": _question,})['result']
res2 = re.sub("\u200c", " ", res2)
res2 = re.sub("\n", " ", res2)
res2 = re.sub("\xa0", " ", res2)
print(res2)



> Entering new StuffDocumentsChain chain...

> Finished chain.
 با توجه به متن زیر به پرسش پاسخ بده: متن: �ه قرار گیرد.  در نهایت، انتخاب پیام رسان مناسب به نیازها و اولویت های کاربران بستگی دارد. با وجود چالش ها و موانع، ایتا میتواند به عنوان یک پیام رسان بومی و امن، جایگاه خود را در میان سایر نرم افزارها پیدا کند و به بهبود کیفیت ارتباطات داخلی کمک نماید. همچنین، با توجه به پتانسیل های بالای ایتا، انتظار میرود که در آینده با توسعه امکانات و رفع معایب موجود، به یکی از رقبای جدی پیام رسان های جهانی تبدیل شود. پرسش: یکی از یزرگ ترین مزایای ایتا چیست؟ پاسخ را به زبان فارسی روان بده: یکی از بزرگترین مزایای ایتا این است که به عنوان یک پیام رسان بومی و امن، جایگاه خود را در میان سایر نرم افزارها پیدا می کند و به بهبود کیفیت ارتباطات داخلی کمک می کند.


In [30]:
_question = "تعداد کاربران ایتا در مقایسه با دیگر پیام رسان‌ها چگونه است؟"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
res2 = qa({"query": _question,})['result']
res2 = re.sub("\u200c", " ", res2)
res2 = re.sub("\n", " ", res2)
res2 = re.sub("\xa0", " ", res2)
print(res2)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




> Entering new StuffDocumentsChain chain...

> Finished chain.
 با توجه به متن زیر به پرسش پاسخ بده: متن: �ها محدودیت هایی دارد. به عنوان مثال، امکانات تجاری و تبلیغاتی که در تلگرام وجود دارد، در ایتا کمتر توسعه یافته است.  تعداد کاربران ایتا در مقایسه با دیگر پیام رسان ها هنوز پایین است. این موضوع میتواند باعث محدودیت در برقراری ارتباط با افرادی که از ایتا استفاده نمی کنند، شود. در دنیای امروز، ارتباطات با دوستان و خانواده در پیام رسان های مختلف بسیار مهم است و کمبود کاربران میتواند به تجربه کاربری آسیب بزند.  مانند هر نرم افزار دیگری، ایتا نیز ممکن است در برخی مواقع با نقص های فنی مواجه شود که میتواند تجربه کاربری را تحت تأثیر قرار دهد. به روز رسانی های ناکافی یا مشکلات در سرورها میتواند موجب نارضایتی کاربران شود.  در مقایسه ایتا با سایر پیام رسان ها، مشخص می شود که هر یک از این نرم افزارها نقاط قوت و ضعف خاص خود را دارند. پیام رسان هایی مانند تلگرام و واتس اپ از لحاظ تعداد کاربران و امکانات بسیار پیشرفته تر هستند. این نرم افزارها دارای قابلیت هایی مانند تماس ویدئو� پرسش: تعداد کار